1.   train・dev・testのコーパスを目視で整える
2.   neologdnで正規化し、sudachiで単語分割する
3.   LightGBMとSVRの入力のために、分割した単語をTf-idfでベクトル化
4.   LSTMとLightGBMとSVRで学習し、学習結果をファイルに出力
5.  出力したファイルをアンサンブル(平均)する

In [ ]:
import numpy
numpy.set_printoptions(threshold=numpy.inf)

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path1 = "/content/drive/MyDrive/研究室コンペ/data/"
path2 = "/content/drive/MyDrive/研究室コンペ/self_data/"
path3 = "/content/drive/MyDrive/研究室コンペ/self_data2/"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from mlxtend.plotting import plot_decision_regions

In [ ]:
def load_data(openfile):
    with open(openfile, 'r') as f:
        text = f.read().split("\n")
        text = text[:-1] # train_textの最後に''があるため削除
    return text

In [ ]:
# データの読み込み
train_text = load_data( path2 + "text.train.txt")
dev_text = load_data( path2 + "text.dev.txt") # 検証用
test_text = load_data( path2 + "text.test.txt") # 提出用

# ラベルの読み込み
train_label = load_data( path2 + "label.train.txt")
dev_label = load_data( path2 + "label.dev.txt")

In [ ]:
#sudachiをインストール
! pip install sudachipy
! pip install sudachidict_core #単語分割用の辞書
! pip install sudachidict_full
! pip install sudachidict_small

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sudachidict_core: filename=SudachiDict_core-20230110-py3-none-any.whl size=71665396 sha256=5b93ef4834e09dea12e670b52edfad31135da17975c4fad0431bc150bb8c1505
  Stored in directory: /root/.cache/pip/wheels/53/fb/7b/5a2f84cba933a120cefe3587a01ed20dbc9a033a6bb43672e7
Successfully built sudachidict_core
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sudachidict_full: filename=SudachiDict_full-20230110-py3-none-any.whl size=126831894 sha256=27589061be18ad99f30ea1b5f3407bc395543a2f98a7165b753ec4d484e20e1d
  Stored in directory: /root/.cache/pip/wheels

In [ ]:
#sudachiによる正規化を行う関数を定義
from sudachipy import Dictionary
from sudachipy import SplitMode
tokenizer = Dictionary(dict="small").create() #dict="full"

def sudachi(text):
    after = list()
    for token in tokenizer.tokenize(text, SplitMode.A):
        word = token.normalized_form() # 正規化あり
        pos = " ".join(token.part_of_speech())
        
        if word.isnumeric():
            word = '0'

        after.append(word)
        
    return after

In [ ]:
! pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neologdn: filename=neologdn-0.5.1-cp38-cp38-linux_x86_64.whl size=208738 sha256=a79da9009223823b54d69d367ce9ad4d90aaec7a01c52d6a0d1d369fbd94e14d
  Stored in directory: /root/.cache/pip/wheels/3e/db/10/b3b26caa63c5da86ea3a25043cc4379a66bb3dd30d6f060a37
Successfully built neologdn


In [ ]:
# 単語分割
import neologdn

train_tokenize = [] 
dev_tokenize = []
test_tokenize = []

def tokenize(infile, outfile):
    for i in range(len(infile)):
        #outfile.append(sudachi(infile[i]))   #正規化なし
        outfile.append(sudachi(neologdn.normalize(infile[i])))    #正規化あり

tokenize(train_text, train_tokenize)
tokenize(dev_text, dev_tokenize)
tokenize(test_text, test_tokenize)

# ファイルに書き込み
def writefile(infile, outfile):
    with open(outfile, 'w') as f:
        for i, wordlist in enumerate(infile):
            f.write(" ".join([str(word) for word in wordlist]) + '\n')

writefile(train_tokenize, "train.txt")
writefile(dev_tokenize, "dev.txt")
writefile(test_tokenize, "test.txt")

In [ ]:
with open("train.txt", 'r') as f:
    traintext = f.read().split("\n")
    traintext = traintext[:-1]
with open("test.txt", 'r') as f:
    testtext = f.read().split("\n")
    testtext = testtext[:-1]
with open("dev.txt", 'r') as f:
    devtext = f.read().split("\n")
    devtext = devtext[:-1]

#LSTM

In [ ]:
! pip install torch==1.6.0
! pip install torchtext==0.7.0
! pip install pytorch-lightning==1.0.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.8/748.8 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.6.0 which is incompatible.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.6.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.6.0 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchtext.data import Example, Field, Dataset, BucketIterator

In [ ]:
y_train =  np.array([x for x in list(map (int, train_label))]) #ラベルを0始まりに
y_dev =  np.array([x for x in list(map (int, dev_label))])

label2id = dict()
for label in sorted(y_train):
    label2id[label] = len(label2id)-1 # ラベルを0始まりに

label2id

{-2: 0, -1: 1, 0: 2, 1: 3, 2: 4}

In [ ]:
#データの読み込み

seed = 64
torch.manual_seed(seed)

batch_size = 2

text_field = Field(sequential=True, use_vocab=True) 
label_field = Field(sequential=False, use_vocab=False, is_target=True)
fields = [("x", text_field), ("t", label_field)]

def load_corpus(text, labels):
    examples = list()
    for line, label in zip(text,labels) :
        word_list = line.strip()
        label_id = label2id[label]
        examples.append(Example.fromlist([word_list, label_id], fields))
    return Dataset(examples, fields)

dataset_train = load_corpus(traintext, y_train)
dataset_val = load_corpus(devtext,y_dev)
y_test = [0] * 2500
dataset_test = load_corpus(testtext, y_test)

# 語彙を登録（訓練データに含まれる単語にIDを割り振る）
text_field.build_vocab(dataset_train, min_freq=3) # 3回以上出てくる単語だけ学習に使う

# データセットオブジェクトからデータローダーを作成
dataloader_train = BucketIterator(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = BucketIterator(dataset_val, batch_size=batch_size, shuffle=False) # ラベルの偏りはどうでもいいためシャッフルしない
dataloader_test = BucketIterator(dataset_test, batch_size=1, shuffle=False) # ラベルの偏りはどうでもいいためシャッフルしな

/usr/local/lib/python3.8/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.8/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.8/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release

In [ ]:
lr = 0.001
n_embed = 64  # 単語ベクトルの次元 
n_hidden = 128  # 文ベクトルの次元
n_layers = 2  # RNN層の数
dropout = 0.15
bidirectional = True

In [ ]:
# RNNによるテキスト分類

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

class RNN(pl.LightningModule):

    # 埋め込み層, 隠れ層, 全結合層の定義   (ニューラルネットワークの形)
    def __init__(self, n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional):
        super(RNN, self).__init__()
        self.embed = nn.Embedding(num_embeddings=n_input, embedding_dim=n_embed, padding_idx=1) # 埋め込み層　nn.Embedding:単語を単語ベクトルに．n_input:モデルに何単語学習させるか．embedding_dim：何次元の単語ベクトルか
        self.lstm = nn.LSTM(input_size=n_embed, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional) # input_size：入力が何次元か．hidden_size：RNN(h)が何次元か．num_layers：RNNが何層か．bidirectional：双方向か片方向か
        self.fc = nn.Linear(in_features=n_hidden * (2 if bidirectional==True else 1), out_features=n_output) # 全結合層 1層のニューラルネットワークを作る　out_features：ラベルの種類(negative.positiveなどの5種類) 両方向のとき，入力*2
        self.drop1 = nn.Dropout(dropout)
        self.drop2 = nn.Dropout(dropout)
 
    # 順伝播
    def forward(self, x):
        y = self.embed(x)
        y = self.drop1(y)
        o, (h, c) = self.lstm(y) # embed：単語ベクトルを作る ->それをlstmに入れる  o, h, c  o(上に出てくるh)とh(横に出ていく・再入力されるh)はおなじ
        y = self.drop1(o)
        return self.fc(o[-1]) #oの一番最後は全体を表している 

    # 訓練用データのバッチを受け取って損失を計算
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("train_loss", loss)
        return loss
    
    # 検証用データのバッチを受け取って損失を計算
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("val_loss", loss)

    # 評価用データのバッチを受け取って分類の正解率を計算
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        y = torch.argmax(y, dim=1)
        accuracy = torch.sum(t == y).item() / (len(y) * 1.0)
        self.log("test_acc", accuracy)

    # 損失関数を設定
    def lossfun(self, y, t):
        return F.cross_entropy(y, t) # kappa_loss(y, t)

    # 最適化手法を設定
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=lr)

In [ ]:
# 訓練

# モデルの保存用ディレクトリがすでにあり、新たに訓練する場合は、そのディレクトリを消す
! rm -r model*

n_input = len(text_field.vocab)
n_output = len(label2id)  # 出力ベクトルの次元（=ラベルの種類数）

rnn1 = RNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional) # これを学習させていく

# 訓練中にモデルを保存するための設定
checkpoint = pl.callbacks.ModelCheckpoint(
    # 検証用データにおける損失が最も小さいモデルを保存する
    monitor="val_loss", mode="min", save_top_k=1,
    # モデルファイル（重みのみ）を "model" というディレクトリに保存する
    save_weights_only=True, dirpath="model1/"
)

# 訓練
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=3, verbose=False, mode="min")
trainer = pl.Trainer(gpus=1, callbacks=[checkpoint, early_stop_callback]) 
trainer.fit(rnn1, dataloader_train, dataloader_val)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

rm: cannot remove 'model*': No such file or directory


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | embed | Embedding | 668 K 
1 | lstm  | LSTM      | 593 K 
2 | fc    | Linear    | 1.3 K 
3 | drop1 | Dropout   | 0     
4 | drop2 | Dropout   | 0     
INFO:lightning:
  | Name  | Type      | Params
------------------------------------
0 | embed | Embedding | 668 K 
1 | lstm  | LSTM      | 593 K 
2 | fc    | Linear    | 1.3 K 
3 | drop1 | Dropout   | 0     
4 | drop2 | Dropout   | 0     


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model1/epoch=1.ckpt
ベストモデルの検証用データにおける損失:  tensor(1.3956, device='cuda:0')


In [ ]:
# devに対するQWK
from sklearn.metrics import cohen_kappa_score

model = RNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional) # これを学習させていく
checkpoint_path = checkpoint.best_model_path
best_model = model.load_from_checkpoint(checkpoint_path, n_input=n_input, n_embed=n_embed, n_hidden=n_hidden, n_layers= n_layers, n_output=n_output, dropout=dropout, bidirectional=bidirectional)


dataloader_val_ = BucketIterator(dataset_val, batch_size=1, shuffle=False) #kappa計算用のデータローダー

y_dev_ =  np.array([x+2 for x in list(map (int, dev_label))])

# 正解率の計算
def test_model(test_loader):
    device = torch.device("cpu")

    with torch.no_grad():

        accs = [] # 各バッチごとの結果格納用
        preds = []

        for batch in test_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = best_model(x).to(device)

            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            preds.append(y_label.item()) 

    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    qwk = cohen_kappa_score(y_dev_, preds, weights='quadratic')
    print('QWK=', qwk)

# テストデータで結果確認
test_model(dataloader_val_)

Accuracy: 36.5%
QWK= 0.40223814890572696



```
seed 42
Accuracy: 35.3%
Std: 0.4779
QWK= 0.44142112494024854
```


In [ ]:
# 提出用データに対して推論
model = RNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional) # これを学習させていく
checkpoint_path = checkpoint.best_model_path
best_model = model.load_from_checkpoint(checkpoint_path, n_input=n_input, n_embed=n_embed, n_hidden=n_hidden, n_layers= n_layers, n_output=n_output, dropout=dropout, bidirectional=bidirectional)

def test_model(test_loader):
    f = open("eval_seed"+str(seed)+".txt", "w")
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device("cpu")

    with torch.no_grad():

        accs = [] # 各バッチごとの結果格納用
        preds = []

        for batch in test_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = best_model(x).to(device)

            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            preds.append(y_label.item()-2) 

            f.write(str(y_label.item()-2))
            f.write("\n")

    f.close()
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    std_acc = torch.tensor(accs).std()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    print('Std: {:.4f}'.format(std_acc))

# テストデータで結果確認
test_model(dataloader_test)

Accuracy: 43.3%
Std: 0.4956


#LightGBM + Optuna

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#sudachiによる正規化を行う関数を定義
from sudachipy import Dictionary
from sudachipy import SplitMode
tokenizer = Dictionary(dict="small").create() #dict="full"

def sudachi(text):
    after = list()
    for token in tokenizer.tokenize(text, SplitMode.C):
        # word = token.surface() # 正規化なし
        word = token.normalized_form() # 正規化あり
        pos = " ".join(token.part_of_speech())
        pos = pos.split(" ")
        
        if word.isnumeric():
            word = '0'

        after.append(word)

    return after

In [ ]:
# 単語分割
import neologdn

train_tokenize = [] 
dev_tokenize = []
test_tokenize = []

def tokenize(infile, outfile):
    for i in range(len(infile)):
        #outfile.append(sudachi(infile[i]))   #正規化なし
        outfile.append(sudachi(neologdn.normalize(infile[i])))    #正規化あり

tokenize(train_text, train_tokenize)
tokenize(dev_text, dev_tokenize)
tokenize(test_text, test_tokenize)

def writefile(infile, outfile):
    with open(outfile, 'w') as f:
        for i, wordlist in enumerate(infile):
            f.write(" ".join([str(word) for word in wordlist]) + '\n')

writefile(train_tokenize, "train.txt")
writefile(dev_tokenize, "dev.txt")
writefile(test_tokenize, "test.txt")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import random

with open("train.txt", 'r') as f:
    traintext = f.read().split("\n")
    traintext = traintext[:-1]
with open("test.txt", 'r') as f:
    testtext = f.read().split("\n")
    testtext = testtext[:-1]
with open("dev.txt", 'r') as f:
    devtext = f.read().split("\n")
    devtext = devtext[:-1]

vectorizer = TfidfVectorizer(smooth_idf=True, analyzer='char', norm='l1') 

x_train = vectorizer.fit_transform(traintext)
x_test = vectorizer.transform(testtext)
x_dev = vectorizer.transform(devtext)

In [ ]:
print("x_trainの形状：", x_train.shape)
print("x_devの形状：", x_dev.shape)
print("x_testの形状：", x_test.shape)

y_train =  np.array(list(map (int, train_label)))
y_dev =  np.array(list(map (int, dev_label)))
print("y_trainの形状：", y_train.shape)    
print("y_devの形状：", y_dev.shape)

x_trainの形状： (30000, 3231)
x_devの形状： (2500, 3231)
x_testの形状： (2500, 3231)
y_trainの形状： (30000,)
y_devの形状： (2500,)


In [ ]:
#Oputunaで閾値探索するためのclass
def cal_qwk(y_true, y_pred):
    """QWK (Quadratic Weighted Kappa) を計算する関数"""
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')
    
class OptunaRounder:
    """Optuna を使って QWK の最適な閾値を探索するクラス"""

    def __init__(self, y_true_, y_pred):
        # 真のラベル
        self.y_true_ = y_true_
        # print(y_true)
        # 予測したラベル
        self.y_pred = y_pred
        # print(y_pred)
        # ラベルの種類
        self.labels = np.unique(y_true_)
    
    def adjust(self, y_pred_, thresholds):
        """閾値にもとづいて予測を補正するメソッド"""
        opt_y_pred = pd.cut(y_pred_, [-np.inf] + thresholds + [np.inf], labels=self.labels)
        # print("関数:", opt_y_pred)
        return opt_y_pred
        

    def __call__(self, trial):
        """最大化したい目的関数"""
        # 閾値を Define by run で追加していく
        thresholds = []
        # ラベルの数 - 1 が必要な閾値の数になる
        for i in range(len(self.labels) - 1):
            # 閾値の下限 (既存の最大 or ラベルの最小値)
            low = max(thresholds) if i > 0 else min(self.labels)
            # 閾値の上限 (ラベルの最大値)
            high = max(self.labels)
            # 閾値の候補を追加する
            t = trial.suggest_uniform(f't{i}', low, high)
            thresholds.append(t)

        # 閾値の候補を元に QWK を計算する
        opt_y_pred_ = self.adjust(self.y_pred, thresholds)
        # print("QWK:",opt_y_pred_)
        return cal_qwk(self.y_true_, opt_y_pred_)

# https://blog.amedama.jp/entry/optuna-qwk-optimization

In [ ]:
from sklearn.metrics import cohen_kappa_score
def lgb_custom_metric_qwk_regression(preds, data):
    """LightGBM のカスタムメトリックを計算する関数

    回帰問題として解いた予測から QWK を計算する"""
    # 正解ラベル
    y_true = data.get_label()
    # 予測ラベル
    y_pred = np.clip(preds, -2, 2) # 単純に予測値を clip 
    y_pred = [int(Decimal(str(num)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)) for num in y_pred] # 四捨五入

    # QWK を計算する
    return 'qwk', cohen_kappa_score(y_true, y_pred, weights='quadratic'), True
# https://blog.amedama.jp/entry/optuna-qwk-optimization

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.6 MB/s eta 0:00:00


In [ ]:
#Optunaでパラメータ探索 

from optuna.integration import lightgbm as lgb
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN

###### ここからがLightGBMの実装 ###### lower()
dtrain = lgb.Dataset(x_train, label=y_train)
ddev = lgb.Dataset(x_dev,label= y_dev)

seed = 109

# 使用するパラメータ
params = {'objective': 'regression',  # 最小化させるべき損失関数
         'metric': 'rmse',#'lgb_custom_metric_qwk_regression',  # 学習時に使用する評価指標
         'random_state': seed,  # 乱数シード
         'boosting_type': 'gbdt',  # boosting_type
         'verbose': -1
          }
verbose_eval = 0  # この数字を1にすると学習時のスコア推移がコマンドライン表示される
# https://qiita.com/c60evaporator/items/2b7a2820d575e212bcf4

# early_stoppingを指定してLightGBM学習　回帰
gbm = lgb.train(params, dtrain,
                valid_sets=[ddev],  # early_stoppingの評価用データ
                #feval=lgb_custom_metric_qwk_regression,
                num_boost_round=10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
                callbacks=[lgb.early_stopping(stopping_rounds=32, verbose=True)] # early_stopping用コールバック関数
                )
best_params = gbm.params
print("Best params:", best_params)
# https://qiita.com/c60evaporator/items/2b7a2820d575e212bcf4

[I 2023-02-13 18:22:59,362] A new study created in memory with name: no-name-db6806de-f964-47f1-a2bb-ce271396c068

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 32 rounds.



feature_fraction, val_score: 1.054274:  14%|#4        | 1/7 [00:28<02:53, 28.90s/it][I 2023-02-13 18:23:28,294] Trial 0 finished with value: 1.0542737753472917 and parameters: {'feature_fraction': 0.6}. Best is trial 0 with value: 1.0542737753472917.

feature_fraction, val_score: 1.054274:  14%|#4        | 1/7 [00:28<02:53, 28.90s/it]

Early stopping, best iteration is:
[181]	valid_0's rmse: 1.05427



feature_fraction, val_score: 1.054146:  29%|##8       | 2/7 [00:54<02:15, 27.01s/it][I 2023-02-13 18:23:53,975] Trial 1 finished with value: 1.0541464074674738 and parameters: {'feature_fraction': 1.0}. Best is trial 1 with value: 1.0541464074674738.

feature_fraction, val_score: 1.054146:  29%|##8       | 2/7 [00:54<02:15, 27.01s/it]

...

Early stopping, best iteration is:
[168]	valid_0's rmse: 1.04663



min_data_in_leaf, val_score: 1.046628:  80%|########  | 4/5 [00:59<00:15, 15.85s/it][I 2023-02-13 18:46:24,337] Trial 63 finished with value: 1.0466283751831178 and parameters: {'min_child_samples': 10}. Best is trial 60 with value: 1.0466283751831178.

min_data_in_leaf, val_score: 1.046628:  80%|########  | 4/5 [00:59<00:15, 15.85s/it]

Early stopping, best iteration is:
[168]	valid_0's rmse: 1.04663



min_data_in_leaf, val_score: 1.046628: 100%|##########| 5/5 [01:15<00:00, 15.10s/it]

Early stopping, best iteration is:
[168]	valid_0's rmse: 1.04663
Best params: {'objective': 'regression', 'metric': 'rmse', 'random_state': 109, 'boosting_type': 'gbdt', 'verbose': -1, 'feature_pre_filter': False, 'lambda_l1': 2.1006326378056314e-05, 'lambda_l2': 1.7524210532753124e-08, 'num_leaves': 38, 'feature_fraction': 0.4, 'bagging_fraction': 0.6334180791467596, 'bagging_freq': 2, 'min_child_samples': 20}


In [ ]:
#探索したパラメータで学習
import lightgbm as lgb

dtrain = lgb.Dataset(x_train, label=y_train)
ddev = lgb.Dataset(x_dev,label= y_dev)

# 使用するパラメータ
params = best_params
verbose_eval = 0  # この数字を1にすると学習時のスコア推移がコマンドライン表示される
# https://qiita.com/c60evaporator/items/2b7a2820d575e212bcf4

# early_stoppingを指定してLightGBM学習　回帰
gbm = lgb.train(params, dtrain,
                valid_sets=[ddev],  # early_stoppingの評価用データ
                # feval=lgb_custom_metric_qwk_regression,
                num_boost_round=10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
                callbacks=[lgb.early_stopping(stopping_rounds=32, verbose=True)] # early_stopping用コールバック関数
                )

# https://qiita.com/c60evaporator/items/2b7a2820d575e212bcf4

[1]	valid_0's rmse: 1.19165
Training until validation scores don't improve for 32 rounds.
[2]	valid_0's rmse: 1.18369
[3]	valid_0's rmse: 1.17632
[4]	valid_0's rmse: 1.16965
[5]	valid_0's rmse: 1.16285
[6]	valid_0's rmse: 1.15801
[7]	valid_0's rmse: 1.15151
[8]	valid_0's rmse: 1.14688
[9]	valid_0's rmse: 1.14434
[10]	valid_0's rmse: 1.1396
[11]	valid_0's rmse: 1.13612
[12]	valid_0's rmse: 1.13361
[13]	valid_0's rmse: 1.13069
[14]	valid_0's rmse: 1.1271
[15]	valid_0's rmse: 1.12418
[16]	valid_0's rmse: 1.12137
[17]	valid_0's rmse: 1.11906
[18]	valid_0's rmse: 1.1171
[19]	valid_0's rmse: 1.11406
[20]	valid_0's rmse: 1.112
[21]	valid_0's rmse: 1.11008
[22]	valid_0's rmse: 1.10896
[23]	valid_0's rmse: 1.10638
[24]	valid_0's rmse: 1.10368
[25]	valid_0's rmse: 1.10264
[26]	valid_0's rmse: 1.10025
[27]	valid_0's rmse: 1.09795
[28]	valid_0's rmse: 1.0957
[29]	valid_0's rmse: 1.09458
[30]	valid_0's rmse: 1.09327
[31]	valid_0's rmse: 1.09091
[32]	valid_0's rmse: 1.08899
[33]	valid_0's rmse: 1.08

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Optuna を使って QWK の閾値を最適化
import optuna

y_pred = gbm.predict(x_dev)
y_pred = np.clip(y_pred, -2, 2)
y_pred_dev = [int(Decimal(str(num)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)) for num in y_pred]
# print(type(y_dev))
objective = OptunaRounder(y_dev, y_pred_dev)
# print(y_pred_)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=100)

# 見つけた閾値
best_thresholds = sorted(study.best_params.values())
print(f'Optimized thresholds: {best_thresholds}')

[I 2023-02-13 18:47:12,916] A new study created in memory with name: no-name-b72069b9-1f6b-4d4f-a002-55e4b2b265f9
<ipython-input-33-d7407117602e>:37: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  t = trial.suggest_uniform(f't{i}', low, high)
[I 2023-02-13 18:47:12,927] Trial 0 finished with value: 0.010384373668737568 and parameters: {'t0': 1.2855961548787351, 't1': 1.8280949980789245, 't2': 1.9017821460464197, 't3': 1.9182631050831989}. Best is trial 0 with value: 0.010384373668737568.
<ipython-input-33-d7407117602e>:37: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  t = trial.suggest_uniform(f't{i}', low, high)
[I 2023-02-13 18:47:12,934] Trial 1 fi

Optimized thresholds: [-0.7459242942134579, -0.01656552952241752, 0.29354630011163285, 0.6261019444219256]


In [ ]:
# 閾値を最適化する前の QWK
y_pred = gbm.predict(x_dev)
y_pred = np.clip(y_pred, -2, 2)
y_pred = [int(Decimal(str(num)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)) for num in y_pred]
qwk = cohen_kappa_score(y_dev,y_pred, weights='quadratic')

# 閾値を最適化した場合の QWK
optimized_oof_y_pred = objective.adjust(y_pred_dev, best_thresholds)
optimized_oof_qwk = cal_qwk(y_dev, optimized_oof_y_pred)

# 書き込み
f = open("LightGBM_Oputuna_dev_small_C_seed"+str()+".txt", "w")
for labeldata in optimized_oof_y_pred:
    f.write(str(labeldata))
    f.write("\n")
f.close()

print('最適化前のQWK=', qwk)
print('最適化後のQWK=',optimized_oof_qwk)

最適化前のQWK= 0.34680632275724144
最適化後のQWK= 0.4281620024285576


In [ ]:
# 提出データに対して推論
y_pred = gbm.predict(x_test)
y_pred = np.clip(y_pred, -2, 2)

y_preds = objective.adjust(y_pred, best_thresholds)

# 書き込み
f = open("LightGBM_Oputuna_eval_small_C_seed"+str()+".txt", "w")
for labeldata in y_preds:
    f.write(str(labeldata))
    f.write("\n")
f.close()

#SVR

In [ ]:
#sudachiによる正規化を行う関数を定義
from sudachipy import Dictionary
from sudachipy import SplitMode
tokenizer = Dictionary(dict="full").create() #dict="small"

def sudachi(text):
    after = list()
    for token in tokenizer.tokenize(text, SplitMode.C):
        # word = token.surface() # 正規化なし
        word = token.normalized_form() # 正規化あり
        pos = " ".join(token.part_of_speech())
        
        if word.isnumeric():
            word = '0'

        after.append(word)

    return after

In [ ]:
import neologdn

train_tokenize = [] 
dev_tokenize = []
test_tokenize = []

def tokenize(infile, outfile):
    for i in range(len(infile)):
        #outfile.append(sudachi(infile[i]))   #正規化なし
        outfile.append(sudachi(neologdn.normalize(infile[i])))    #正規化あり

tokenize(train_text, train_tokenize)
tokenize(dev_text, dev_tokenize)
tokenize(test_text, test_tokenize)

def writefile(infile, outfile):
    with open(outfile, 'w') as f:
        for i, wordlist in enumerate(infile):
            f.write(" ".join([str(word) for word in wordlist]) + '\n')
            
writefile(train_tokenize, "train.txt")
writefile(dev_tokenize, "dev.txt")
writefile(test_tokenize, "test.txt")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import random

with open("train.txt", 'r') as f:
    traintext = f.read().split("\n")
    traintext = traintext[:-1]
with open("test.txt", 'r') as f:
    testtext = f.read().split("\n")
    testtext = testtext[:-1]
with open("dev.txt", 'r') as f:
    devtext = f.read().split("\n")
    devtext = devtext[:-1]
    
vectorizer = TfidfVectorizer(smooth_idf=True, analyzer='char', norm='l1') 

x_train = vectorizer.fit_transform(traintext)
x_test = vectorizer.transform(testtext)
x_dev = vectorizer.transform(devtext)

In [ ]:
print("x_trainの形状：", x_train.shape)
print("x_devの形状：", x_dev.shape)
print("x_testの形状：", x_test.shape)

y_train =  np.array(list(map (int, train_label)))
y_dev =  np.array(list(map (int, dev_label)))
print("y_trainの形状：", y_train.shape)    
print("y_devの形状：", y_dev.shape)

x_trainの形状： (30000, 3248)
x_devの形状： (2500, 3248)
x_testの形状： (2500, 3248)
y_trainの形状： (30000,)
y_devの形状： (2500,)


In [ ]:
from sklearn.metrics import cohen_kappa_score
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
best_qwk = 0
best_c = 0
for c in [11, 12, 13, 14]: # 0.1, 1, 8, 10, 11, 12, 13, 14, 15, 20, 100]:
    model = SVR(C=c, kernel='rbf')
    model.fit(x_train, y_train)
    y_pred = model.predict(x_dev)
    y_pred = [int(Decimal(str(num)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)) for num in y_pred] #y_pred =  np.clip(y_pred, -2, 2)

    qwk = cohen_kappa_score(y_dev, y_pred, weights='quadratic')
    if qwk > best_qwk:
        best_qwk = qwk
        best_c = c
    print("QWK = %f  C = %s" % (qwk, str(c)))
print("最適なハイパーパラメタは C = %s" % str(best_c))



```
QWK = 0.244240  C = 0.1
QWK = 0.342857  C = 1
QWK = 0.358512  C = 8
QWK = 0.360366  C = 10
QWK = 0.361996  C = 11
QWK = 0.358468  C = 12
QWK = 0.355736  C = 15
QWK = 0.355241  C = 20
QWK = 0.354434  C = 100
最適なハイパーパラメタは C = 11
```



In [ ]:
def cal_qwk(y_true, y_pred):
    """QWK (Quadratic Weighted Kappa) を計算する関数"""
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')
    
class OptunaRounder:
    """Optuna を使って QWK の最適な閾値を探索するクラス"""

    def __init__(self, y_true_, y_pred):
        # 真のラベル
        self.y_true_ = y_true_
        # print(y_true)
        
        # 予測したラベル
        self.y_pred = y_pred
        # print(y_pred)
        # ラベルの種類
        self.labels = np.unique(y_true_)
    
    def adjust(self, y_pred_, thresholds):
        """閾値にもとづいて予測を補正するメソッド"""
        opt_y_pred = pd.cut(y_pred_, [-np.inf] + thresholds + [np.inf], labels=self.labels)
        # print("関数:", opt_y_pred)
        return opt_y_pred
        

    def __call__(self, trial):
        """最大化したい目的関数"""
        # 閾値を Define by run で追加していく
        thresholds = []
        # ラベルの数 - 1 が必要な閾値の数になる
        for i in range(len(self.labels) - 1):
            # 閾値の下限 (既存の最大 or ラベルの最小値)
            low = max(thresholds) if i > 0 else min(self.labels)
            # 閾値の上限 (ラベルの最大値)
            high = max(self.labels)
            # 閾値の候補を追加する
            t = trial.suggest_uniform(f't{i}', low, high)
            thresholds.append(t)

        # 閾値の候補を元に QWK を計算する
        opt_y_pred_ = self.adjust(self.y_pred, thresholds)
        # print("QWK:",opt_y_pred_)
        return cal_qwk(self.y_true_, opt_y_pred_)


In [ ]:
!pip install optuna

In [ ]:
import optuna
from sklearn.svm import SVR

model = SVR(C=best_c, kernel='rbf')
model.fit(x_train, y_train)
y_pred = model.predict(x_dev)

y_pred_dev = [int(Decimal(str(num)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)) for num in y_pred]

objective = OptunaRounder(y_dev, y_pred_dev)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, timeout=100)

# 見つけた閾値
best_thresholds = sorted(study.best_params.values())
print(f'Optimized thresholds: {best_thresholds}')


In [ ]:
# 閾値を最適化した場合の QWK
optimized_oof_y_pred = objective.adjust(y_pred_dev, best_thresholds)

optimized_oof_qwk = cal_qwk(y_dev, optimized_oof_y_pred)

print('閾値最適化前のQWK', cal_qwk(y_dev, y_pred))
print('閾値最適化後のQWK', optimized_oof_qwk)

In [ ]:
# 提出データに対して推論

y_pred = model.predict(x_test)

# 最適化した閾値をアンサンブル結果に反映
y_preds = objective.adjust(y_pred, best_thresholds)

# 書き込み
f = open("SVR_eval_oputuna_C"+str(best_c)+".txt", "w")
for labeldata in y_preds:
    f.write(str(int(labeldata)))
    f.write("\n")
f.close()

#アンサンブル


In [ ]:
enspath = "/content/drive/MyDrive/研究室コンペ/2_アンサンブル/"
enspath2 = "/content/drive/MyDrive/研究室コンペ/2_アンサンブル_QWK/"

# 提出用データに対しての出力のみを保存
# 閾値最適化のためにdevの出力も保存

In [ ]:
def load_data(openfile):
    with open(enspath + openfile, 'r') as f:
        text = f.read().split("\n")
        text = text[:-1] 
    return text

def load_data2(openfile):
    with open(enspath2 + openfile, 'r') as f:
        text = f.read().split("\n")
        text = text[:-1]
    return text

def get_label(result):
    return [int(x) for x in result]

In [ ]:
f0 = 'LightGBM_Oputuna1.txt'
#LSTM
f1 = '提出0.329.txt'
f2 = '提出0.419.txt'
f3 = '提出0.450.txt' #
f4 = '提出0.381.txt'
f5 = '提出0.453.txt' #
f6 = '提出0.455.txt' #
f7 = '提出0.388.txt'
f8 = '提出0.438.txt'  
f9 = '提出0.401C.txt'
f10 = '提出0.405.txt'  
f11 = '提出0.451.txt'  #
f12 = '提出0.411.txt'
f13 = '提出0.414.txt'  
f14 = '提出0.439.txt'  
f15 = '提出linear2rnn1.txt'

# CatBoost+Oputuna
eval_0 = 'CatBoost_Oputuna_eval.txt'
# LSTM
eval_1 = 'eval_seed0_0.421.txt'
eval_2 = 'eval_seed123_0.441.txt'
eval_3 = 'eval_seed1234_0.448.txt'
eval_4 = 'eval_seed2023_0.432.txt'
eval_5 = 'eval_seed32_0.410.txt'
eval_6 = 'eval_seed32_0.442_rnn2.txt'
eval_7 = 'eval_seed42_0.456.txt' #
eval_8 = 'eval_seed64rnn2_0.426.txt'
# LightGBM+Oputuna
eval_9 = 'LightGBM_Oputuna_eval_small_C_seed1234.txt' 
eval_10 = 'LightGBM_Oputuna_eval_small_C_seed32.txt'
eval_11 = 'LightGBM_Oputuna_eval_small_C_seed42.txt'#
eval_12 = 'LightGBM_Oputuna_eval.txt'
# linearSVR
eval_13 = 'linearSVR_eval_oputuna_C150_seed605.txt'
eval_14 = 'linearSVR_eval_oputuna_C160_seed1234.txt'
eval_15 = 'linearSVR_eval_oputuna_C160_seed22.txt'
eval_16 = 'linearSVR_eval_oputuna_C190_seed123.txt'
eval_17 = 'linearSVR_eval_oputuna_C190_seed42.txt'
# SVR
eval_18 = 'SVR_eval_oputuna_C11.txt' #

In [ ]:
# ファイルの読み込み
res0 = load_data(f0)
res1 = load_data(f1)
res2 = load_data(f2)
res3 = load_data(f3)
res4 = load_data(f4)
res5 = load_data(f5)
res6 = load_data(f6)
res7 = load_data(f7)
res8 = load_data(f8)
res9 = load_data(f9)
res10 = load_data(f10)
res11 = load_data(f11)
res12 = load_data(f12)
res13 = load_data(f13)
res14 = load_data(f14)
res15 = load_data(f15)

res0_2 = load_data2(eval_0)
res1_2 = load_data2(eval_1)
res2_2 = load_data2(eval_2)
res3_2 = load_data2(eval_3)
res4_2 = load_data2(eval_4)
res5_2 = load_data2(eval_5)
res6_2 = load_data2(eval_6)
res7_2 = load_data2(eval_7)
res8_2 = load_data2(eval_8)
res9_2 = load_data2(eval_9)
res10_2 = load_data2(eval_10)
res11_2 = load_data2(eval_11)
res12_2 = load_data2(eval_12)
res13_2 = load_data2(eval_13)
res14_2 = load_data2(eval_14)
res15_2 = load_data2(eval_15)
res16_2 = load_data2(eval_16)
res17_2 = load_data2(eval_17)
res18_2 = load_data2(eval_18)

In [ ]:
# ラベルの取り出し
res0 = get_label(res0)
res1 = get_label(res1)
res2 = get_label(res2)
res3 = get_label(res3) #
res4 = get_label(res4)
res5 = get_label(res5) #
res6 = get_label(res6) #
res7 = get_label(res7)
res8 = get_label(res8)
res9 = get_label(res9)
res10 = get_label(res10)
res11 = get_label(res11) #
res12 = get_label(res12)
res13 = get_label(res13)
res14 = get_label(res14)
res15 = get_label(res15)

res0_2 = get_label(res0_2)
res1_2 = get_label(res1_2)
res2_2 = get_label(res2_2)
res3_2 = get_label(res3_2)
res4_2 = get_label(res4_2)
res5_2 = get_label(res5_2)
res6_2 = get_label(res6_2)
res7_2 = get_label(res7_2) #
res8_2 = get_label(res8_2)
res9_2 = get_label(res9_2)
res10_2 = get_label(res10_2)
res11_2 = get_label(res11_2) #
res12_2 = get_label(res12_2)
res13_2 = get_label(res13_2)
res14_2 = get_label(res14_2)
res15_2 = get_label(res15_2)
res16_2 = get_label(res16_2)
res17_2 = get_label(res17_2)
res18_2 = get_label(res18_2) #

In [ ]:
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
    
ens = []
for i in range(len(res0)):
    # label = float(res3[i]+res5[i]+res6[i]+res11[i]+res11_2[i]+res17_2[i]+res18_2[i])/7 #0.524
    # label = float(res6[i]+res7_2[i]+res11_2[i]+res18_2[i])/4 # 0.512
    # label = float(res5[i]+res6[i]+res11[i]+res7_2[i]+res11_2[i]+res18_2[i])/5 #0.522
    label = float(res3[i]+res5[i]+res6[i]+res11[i]+res7_2[i]+res11_2[i]+res18_2[i])/6 #0.529
    label = int(Decimal(str(label)).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
    print(label)
    ens.append(label)
#
# 書き込み
f = open("ensemble_eval.txt", "w")
for labeldata in ens:
    f.write(str(int(labeldata)))
    f.write("\n")
f.close()

1
2
1
2
2
2
-1
-1
1
2
0
2
1
2
0
2
1
2
1
0
0
-1
1
0
0
-2
1
2
2
0
-1
1
0
0
0
2
0
1
0
0
1
0
2
1
1
0
0
0
-1
-1
0
0
0
-2
1
-1
0
0
0
1
0
0
0
0
2
2
0
1
1
-1
1
-1
1
1
0
0
1
1
0
1
-2
2
-1
1
1
-1
1
-1
1
0
1
-1
0
1
0
0
0
1
-1
-1
1
1
0
1
0
0
1
-1
-1
1
-1
-1
1
-1
1
2
0
0
1
-1
2
-2
1
-2
1
1
-1
2
0
0
0
0
1
1
0
0
0
1
-1
1
2
-1
1
-1
0
2
2
2
0
0
2
0
-1
0
1
-2
0
1
0
0
1
0
0
1
-1
0
1
1
1
0
0
0
0
1
0
-1
-1
-1
0
1
0
-1
0
0
-1
-1
1
2
0
1
0
1
2
0
1
1
0
0
0
1
1
2
2
0
0
1
0
-1
0
2
0
1
1
0
0
0
1
1
0
0
1
1
1
0
1
2
2
2
0
0
-2
1
1
0
1
-1
1
-1
1
1
2
0
2
0
1
1
1
-1
1
2
1
-1
2
-2
1
1
1
-1
0
-1
-1
1
0
1
-1
-2
-1
0
0
0
-1
-1
2
0
-1
1
-1
1
-2
-1
0
0
0
0
0
1
2
1
1
2
2
1
0
0
0
-1
0
0
0
1
2
-1
0
1
1
2
0
0
-1
1
1
-1
2
1
0
1
-1
-1
2
-1
0
-2
0
1
-1
1
2
0
-1
2
1
2
0
2
0
2
0
1
2
1
1
0
0
0
-1
-1
2
1
1
1
-1
-2
-1
0
-1
1
1
-1
2
2
0
1
0
-1
-1
0
2
1
-1
2
0
0
1
-1
1
1
0
0
1
-1
1
-1
-1
2
1
0
1
1
1
1
1
2
1
-1
1
1
0
-1
1
0
1
0
1
1
0
-1
0
1
0
-1
0
-1
1
0
0
1
2
0
-1
0
1
-1
0
1
1
2
0
1
-1
-1
2
1
1
0
-1
0
2
0
-1
2
2
-1
0
1
-1
1
1
1
0
-1
1
-2



```
res3+res5+res6+res11+res11_2+res17_2+res18_2 #0.524
res6+res7_2+res11_2+res18_2 # 0.512
res5+res6+res11+res7_2+res11_2+res18_2 #0.522
res3+res5+res6+res11+res7_2+res11_2+res18_2 #0.529
```

